In [863]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity

from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector

Lx, Ly = (4, 4)
L = Lx*Ly
t = .1
latt = qib.lattice.TriangularLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 2.5)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()

X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
I2 = np.array([[1, 0], [0, 1]])


perms_1 = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12]]
perms_2 = [[0, 5, 10, 15, 3, 4, 9, 14, 2, 7, 8, 13, 1, 6, 11, 12], [5, 10, 15, 0, 4, 9, 14, 3, 7, 8, 13, 2, 6, 11, 12, 1]]
perms_3 = [[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15], [4, 8, 12, 0, 5, 9, 13, 1, 6, 10, 14, 2, 7, 11, 15, 3]]

from itertools import product
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np


def eval_energy(config):
    e = 0
    for perm in perms_1+perms_2+perms_3:
        for j in range(len(perm)//2):
            e += 1 if config[perm[2*j]]==config[perm[2*j+1]] else -1
    return e
e = 48
gs = '0'*L
gss = []
bitstrings = [''.join(bits) for bits in product('01', repeat=16)]
for bitstring in bitstrings:
    if eval_energy(bitstring) < e:
        gs = bitstring
        e = eval_energy(bitstring)
    if eval_energy(bitstring) == -16:
        gss.append(bitstring)
ground_states = gss
n = len(ground_states[0])
dim = 2**n
state = np.zeros(dim, dtype=complex)
for s in ground_states:
    index = int(s, 2)
    state[index] = 1 / np.sqrt(len(ground_states))
psi = Statevector(state)

#eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=100)
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=10, v0=psi.data, which='SA')
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

eigenvalues_sort[0]

-43.541843786906114

In [871]:
"""
    Compressed-Controlled Time Evolution Operator that we optimized previously.
"""
import h5py
import sys
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp

Vlists = {}
ts = [0.1, 0.2]
for t in ts:
    Vlist = []
    with h5py.File(f'../triangularTFIM2d/results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers9_niter{10 if g!=1.5 else 5}_rS1_2hloc.hdf5') as f:
        Vlist =  f["Vlist"][:]
        
    perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
                        [perms_2] + [[perms_2[0]], [perms_3[0]]] + [perms_3] + [[perms_3[0]]] 
    perms_ext_reduced = [perms_1]  + [perms_2] + [perms_3]
    control_layers = [0, 2, 3, 5, 6, 8]
    Vlists[t] = Vlist

qcs = {}
for t in ts:
    Vlist = Vlists[t]
    qc = qiskit.QuantumCircuit(L+1)
    qc.x(L)
    for i, V in enumerate(Vlist):
        layer = i
        if i in control_layers:
            if i in [2, 5, 6]:
                continue
            elif i in [0, 8]:
                for perm in perms_extended[layer]:
                    for j in range(L//2):
                        qc.cy(L, L-1-perm[2*j])
                        qc.cz(L, L-1-perm[2*j+1])
            elif i==3:
                for perm in perms_extended[layer]:
                    for _ in range(L//Lx):
                        for j in range(Ly//2):
                            qc.cx(L, L-1-Lx*_-(2*j + (1 if _%2==0 else 0)))
            
        else:
            for perm in perms_extended[layer]:
                for j in range(L//2):
                    qc.unitary(V, [L-perm[2*j]-1, L-perm[2*j+1]-1])
    qc.x(L)
    qcs[t] = qc

t = 0.1
state = random_statevector(2**L).data
qc_ext1 = qiskit.QuantumCircuit(L+1)
qc_ext1.initialize(state, [i for i in range(L)])
qc_ext1.append(qcs[t].to_gate(), [i for i in range(L+1)])
backend = Aer.get_backend("statevector_simulator")
sv1 = execute(transpile(qc_ext1), backend).result().get_statevector().data
qc_ext2 = qiskit.QuantumCircuit(L+1)
qc_ext2.initialize(state, [i for i in range(L)])
qc_ext2.x(L)
qc_ext2.append(qcs[t].to_gate(), [i for i in range(L+1)])
backend = Aer.get_backend("statevector_simulator")
sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])
exact_v1 = np.kron(ket_0, expm_multiply(1j * t * hamil, state))
exact_v2 = np.kron(ket_1, expm_multiply(-1j * t * hamil, state))
err = 1-(state_fidelity(sv1, exact_v1)+state_fidelity(sv2, exact_v2))/2
err

0.003069126333485439

In [872]:
anc = 2

qc_QPE = qiskit.QuantumCircuit(L+anc, anc)
qc_QPE.x(range(L))
qc_QPE.h(range(L+anc))
for digit in range(anc):
    for _ in range(digit+1):
        qc_QPE.append(qcs[0.1].to_gate(), list(range(L)) + [L+digit])
    #qc_QPE.append(qcs[ts[digit]].to_gate(), list(range(L)) + [L+digit])
iqft = QFT(num_qubits=anc, inverse=True, do_swaps=True)
qc_QPE.append(iqft, range(L+anc-1, L-1, -1))
qc_QPE.measure(range(L+anc-1, L-1, -1), range(anc)[::-1])

backend = Aer.get_backend('aer_simulator')
counts_coarse = execute(transpile(qc_QPE), backend, shots=int(1e3
                                                             )).result().get_counts()
print(counts_coarse)

{'01': 72, '10': 27, '00': 224, '11': 677}


In [873]:
import scipy.sparse as sp
from qiskit.circuit.library import QFT
from qiskit import Aer, execute, transpile

anc = 2

qc_QPE = qiskit.QuantumCircuit(L+anc, anc+1)
qc_QPE.x(range(L))
qc_QPE.h(range(L+anc))
for digit in range(anc):
    #qc_QPE.append(qcs[ts[digit]].to_gate(), list(range(L)) + [L+digit])
    for _ in range(digit+1):
        qc_QPE.append(qcs[0.1].to_gate(), list(range(L)) + [L+digit])
iqft = QFT(num_qubits=anc, inverse=True, do_swaps=True)
qc_QPE.append(iqft, range(L+anc-1, L-1, -1))

backend = Aer.get_backend("statevector_simulator")
sv = execute(transpile(qc_QPE), backend).result().get_statevector().data
op = sp.kron(sp.kron(sp.csr_matrix(rho_1_anc), sp.csr_matrix(rho_1_anc)), sp.eye(2**L, format="csr"))
sv = op @ sv
sv = sv / np.linalg.norm(sv)

print("Fidelity after Projection: ", state_fidelity(sv, np.kron(np.kron(ket_1, ket_1),  eigenvectors_sort[:, 0])))

Fidelity after Projection:  0.9665320853149578


In [874]:
from qiskit.circuit.library import QFT
from qiskit import Aer, execute, transpile

anc = 2

qc_QPE = qiskit.QuantumCircuit(L+anc, anc+1)
#qc_QPE.initialize(ground_state, [i for i in range(L-1, -1, -1)])
#qc_QPE.h([i for i in range(L, L+anc)])
qc_QPE.x(range(L))
qc_QPE.h(range(L+anc))
for digit in range(anc):
    #for _ in range(digit+1):
    #    qc_QPE.append(qcs[0.1].to_gate(), list(range(L)) + [L+digit])
    qc_QPE.append(qcs[ts[digit]].to_gate(), list(range(L)) + [L+digit])
iqft = QFT(num_qubits=anc, inverse=True, do_swaps=True)
qc_QPE.append(iqft, range(L+anc-1, L-1, -1))
qc_QPE.measure(range(L+anc-1, L-1, -1), range(anc)[::-1])
qc_QPE.h(0)
qc_QPE.measure(0, 2)

backend = Aer.get_backend('aer_simulator')
counts = execute(transpile(qc_QPE), backend, shots=int(1e3)).result().get_counts()
print(counts)
x_exp = (counts['011'] - counts['111'])/(counts['011'] + counts['111'])
print("<X> from QPE: ", x_exp)

{'101': 51, '100': 206, '001': 15, '110': 31, '010': 5, '000': 44, '011': 40, '111': 608}
<X> from QPE:  -0.8765432098765432


In [875]:
qc_init = qiskit.QuantumCircuit(L, 1)
qc_init.initialize(ground_state, range(L))

qc_init.h(0)
qc_init.measure(0, 0)
backend = Aer.get_backend('aer_simulator')
counts = execute(transpile(qc_init), backend, shots=int(1e5)).result().get_counts()
x_expectation = (counts['0'] - counts['1'])/(counts['0'] + counts['1'])
print(f"<X0> ≈ {x_expectation:.3f}")

qc_init = qiskit.QuantumCircuit(L, 1)
qc_init.initialize(ground_state, range(L))
qc_init.measure(0, 0)
backend = Aer.get_backend('aer_simulator')
counts = execute(transpile(qc_init), backend, shots=int(1e5)).result().get_counts()
z_expectation = (counts['0'] - counts['1'])/(counts['0'] + counts['1'])
print(f"<Z0> ≈ {z_expectation:.3f}")

corrs = []
for i in range(1, L):
    qc_init = qiskit.QuantumCircuit(L, 2)
    qc_init.initialize(ground_state, range(L))
    qc_init.measure([0, i], [0, 1])
    backend = Aer.get_backend('aer_simulator')
    counts = execute(transpile(qc_init), backend, shots=int(1e5)).result().get_counts()
    xx_vals = []
    for bitstring, count in counts.items():
        b0 = int(bitstring[-1])
        b1 = int(bitstring[-2])
        x0 = 1 if b0 == 0 else -1
        x1 = 1 if b1 == 0 else -1
        xx_vals.extend([x0 * x1] * count)
    zz_expectation = np.mean(xx_vals)
    print(f"C_01 ≈ {zz_expectation - z_expectation**2 :.3f}")
    corrs.append(zz_expectation - z_expectation**2)

<X0> ≈ -0.936
<Z0> ≈ -0.003
C_01 ≈ -0.129
C_01 ≈ 0.030
C_01 ≈ -0.131
C_01 ≈ -0.120
C_01 ≈ -0.127
C_01 ≈ 0.033
C_01 ≈ 0.034
C_01 ≈ 0.027
C_01 ≈ 0.033
C_01 ≈ 0.028
C_01 ≈ 0.029
C_01 ≈ -0.125
C_01 ≈ 0.027
C_01 ≈ 0.034
C_01 ≈ -0.131


In [876]:
corrs

[-0.1285710224,
 0.030408977599999998,
 -0.13135102240000002,
 -0.1202510224,
 -0.1269710224,
 0.032968977600000005,
 0.0339089776,
 0.0272089776,
 0.0330089776,
 0.0277489776,
 0.0288489776,
 -0.1249110224,
 0.026748977599999998,
 0.0338889776,
 -0.13099102240000002]